In [1]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

image_folder_path = '/kaggle/input/segmented-medicinal-leaf-images/Segmented Medicinal Leaf Images'
# Load and Preprocess Images
width = 64  # Set the desired image width for resizing
height = 64  # Set the desired image height for resizing

X = []  # Image data
y = []  # Labels

# Loop through subfolders (classes)
for class_name in os.listdir(image_folder_path):
    class_dir = os.path.join(image_folder_path, class_name)
    
    # Loop through images in each class
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        image = cv2.resize(image, (width, height))  # Resize to a fixed size
        X.append(image.flatten())  # Convert to a 1D feature vector
        y.append(class_name)  # Label is the subfolder (class) name

# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Building
k_neighbors = 3  # You can adjust the number of neighbors as needed
knn_model = KNeighborsClassifier(n_neighbors=k_neighbors)
knn_model.fit(X_train, y_train)

# Model Evaluation
y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.6348773841961853
Classification Report:
                                              precision    recall  f1-score   support

                     Alpinia Galanga (Rasna)       0.50      1.00      0.67         5
            Amaranthus Viridis (Arive-Dantu)       0.41      0.73      0.52        22
        Artocarpus Heterophyllus (Jackfruit)       0.45      0.91      0.61        11
                   Azadirachta Indica (Neem)       1.00      0.56      0.71         9
                       Basella Alba (Basale)       0.52      0.81      0.63        16
            Brassica Juncea (Indian Mustard)       0.50      0.38      0.43         8
                  Carissa Carandas (Karanda)       0.53      0.83      0.65        12
                        Citrus Limon (Lemon)       0.38      0.45      0.42        11
             Ficus Auriculata (Roxburgh fig)       0.50      0.55      0.52        11
               Ficus Religiosa (Peepal Tree)       1.00      0.73      0.85        15
 

In [ ]:
import cv2
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Set the path to your dataset folder on Kaggle
image_folder_path = '/kaggle/input/segmented-medicinal-leaf-images/Segmented Medicinal Leaf Images'

# Set the number of random images you want to select
num_random_images = 2

# Initialize a list to store the paths of the selected images
selected_image_paths = []

# Loop through subfolders (classes)
for class_name in os.listdir(image_folder_path):
    class_dir = os.path.join(image_folder_path, class_name)
    
    # Get a list of image file names in the class directory
    image_files = os.listdir(class_dir)
    
    # Randomly select images from the class
    random_images = random.sample(image_files, num_random_images)
    
    # Create the full paths to the selected images
    selected_image_paths.extend([os.path.join(class_dir, img) for img in random_images])

# Define a function to visualize images with annotations
def visualize_image_with_annotations(image_path, annotations):
    # Load the image
    img = cv2.imread(image_path)
    
    # Create a figure and axis
    fig, ax = plt.subplots()
    
    # Display the image
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Add annotations to the image
    for annotation in annotations:
        ax.add_patch(annotation)
    
    # Add labels to annotations
    for i, annotation in enumerate(annotations):
        x, y = annotation.get_xy()
        ax.text(x, y, f"Annotation {i + 1}", fontsize=12, color='white')
    
    # Remove axis labels
    ax.axis('off')
    
    # Show the image with annotations
    plt.show()

# Display the selected random images with annotations (e.g., rectangles)
for img_path in selected_image_paths:
    # Generate random annotations (e.g., rectangles) for demonstration purposes
    annotations = [
        patches.Rectangle((10, 10), 50, 50, linewidth=2, edgecolor='r', facecolor='none'),
        patches.Rectangle((80, 80), 30, 30, linewidth=2, edgecolor='b', facecolor='none')
    ]
    
    # Visualize the image with annotations
    visualize_image_with_annotations(img_path, annotations)


In [7]:
import cv2
import numpy as np
from PIL import Image
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display

# Define a function to preprocess an uploaded image
def preprocess_uploaded_image(uploaded_image):
    # Convert the uploaded image to grayscale
    img_gray = cv2.cvtColor(np.array(uploaded_image), cv2.COLOR_RGB2GRAY)
    # Resize the image to match the model's input size (64x64)
    img_resized = cv2.resize(img_gray, (64, 64))
    # Flatten the image into a 1D feature vector
    img_flattened = img_resized.flatten()
    return img_flattened

# Define a function to make predictions using the KNN model
def predict_with_knn(image_data):
    return knn_model.predict([image_data])[0]

# Create an upload button for image selection
upload_button = widgets.FileUpload(
    accept='.jpg, .jpeg, .png',  # Specify accepted file types
    multiple=False,  # Allow only a single file to be uploaded
    description='Upload an Image'
)

# Create an output widget for displaying predictions
output = widgets.Output()

# Flag to track if an image has been uploaded
image_uploaded = False

# Define an event handler for the upload button
def on_upload_button_click(change):
    global image_uploaded
    if upload_button.value and not image_uploaded:
        uploaded_image = Image.open(io.BytesIO(upload_button.value[0]['content']))
        preprocessed_image = preprocess_uploaded_image(uploaded_image)
        prediction = predict_with_knn(preprocessed_image)
        
        with output:
            plt.gcf().clear()  # Clear the previous plot
            plt.imshow(uploaded_image)
            plt.axis('off')
            plt.title(f'Predicted Class: {prediction}')
            plt.show()
        
        # Disable the upload button after an image is uploaded
        upload_button.disabled = True
        image_uploaded = True

# Attach the event handler to the upload button
upload_button.observe(on_upload_button_click, names='value')

# Create a VBox layout to arrange widgets vertically
ui = widgets.VBox([upload_button, output])

# Display the user interface
display(ui)


In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Load and Preprocess Images
image_folder_path = '/kaggle/input/segmented-medicinal-leaf-images/Segmented Medicinal Leaf Images'
width = 64
height = 64

X = []
y = []

for class_name in os.listdir(image_folder_path):
    class_dir = os.path.join(image_folder_path, class_name)
    
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (width, height))
        X.append(image.flatten())
        y.append(class_name)

# Step 2: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Model Building with Hyperparameter Tuning
param_grid = {
    'n_estimators': [50, 100, 150],          # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],        # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],        # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],          # Minimum samples required for a leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider for the best split
}

# Create a Random Forest classifier
random_forest_model = RandomForestClassifier(random_state=42)

# Perform Grid Search for hyperparameter tuning
grid_search = GridSearchCV(random_forest_model, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best hyperparameters to build the final model
random_forest_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features='sqrt',
    random_state=42
)

random_forest_model.fit(X_train, y_train)

# Step 4: Model Evaluation
y_pred = random_forest_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.2s


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.2s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.3s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.2s


...

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.6s


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.6s


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


KeyboardInterrupt: 